# Проект для «Викишоп» c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Мы обучим модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построим модель со значением метрики качества *F1* не меньше 0.75. 

## Подготовка

### Импорт библиотек

In [1]:
!pip install transformers
!pip install catboost
!pip install optuna

     |████████████████████████████████| 2.8 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 44.0 MB/s 
     |████████████████████████████████| 3.3 MB 51.0 MB/s 
     |████████████████████████████████| 636 kB 66.1 MB/s 
     |████████████████████████████████| 52 kB 1.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 67.4 MB 73 kB/s 
     |████████████████████████████████| 302 kB 5.2 MB/s 
     |████████████████████████████████| 80 kB 12.7 MB/s 
     |████████████████████████████████| 208 kB 60.2 MB/s 
     |████████████████████████████████| 75 kB 5.7 MB/s 
     |████████████████████████████████| 111 kB 76.5 MB/s 
     |████████████████████████████████| 144 kB 63.9 MB/s 
     |████████████████████████████████| 49 kB 6.9 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11136 sha256=681375217ab21a7f

In [2]:
import numpy as np
import pandas as pd
import torch
import transformers
from transformers import DistilBertTokenizer, DistilBertModel
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import sklearn.svm

import optuna

from google.colab import files


In [3]:
files.upload()

Saving toxic_comments.csv to toxic_comments.csv


### Создадим токенайзер и модель

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

### Прочитаем таблицу

In [ ]:
df_tweets = pd.read_csv('toxic_comments.csv')
df_tweets.info()
df_tweets.head()

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

В таблице 2 столбца. Один столбец *text* с текстами комментаторов, который нам предстоит перевести в признаки для обучения и предсказания нашей модели, а второй столбец *toxic* содержит целевой признак. Что бы хотя бы немного ускорить работу программы, переведём данные в столбце *toxic* из int64 в uint8 

In [ ]:
df_tweets['toxic'] = df_tweets['toxic'].astype('uint8')
df_tweets.info()

Выборку уменьшим, что бы не ждать бескнечно долго рассчётов дальнейшего эмбеддинга.

In [ ]:
df_tweets = df_tweets.sample(2000, replace=True).reset_index(drop=True)

In [ ]:
tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x, truncation=True, add_special_tokens=True))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

### Произведём эмбеддинг

In [ ]:

batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

### Разделим таблицу на выборки

Объединим эмбеддинги, чтобы получить признаки для дальнейшей разбивки на тренировочную и тестовую выборки, а также получим целевой признак из столбца `df_tweets['toxic']`.

In [ ]:
features = np.concatenate(embeddings)

target = df_tweets['toxic']
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=12345)

## Обучим модели

### Модель логистической регрессии

In [ ]:
model_log = LogisticRegression()
model_log.fit(features_train, target_train)
predictions = model_log.predict(features_test)
f1_log = f1_score(target_test, predictions)
print('F1 модели логистической регрессии:', '{:.2f}'.format(f1_log))

### Модель случайного леса

Для подбора гиперпараметров используем библиотеку *optuna*.

In [ ]:
from sklearn.metrics import make_scorer

def objective(trial):
    rf_max_depth = int(trial.suggest_loguniform('rf_max_depth', 2, 20))
    rf_n_estimators = int(trial.suggest_loguniform('rf_n_estimators', 2, 80))
    classifier_obj = RandomForestClassifier(max_depth=rf_max_depth, n_estimators=rf_n_estimators, random_state=12345)
        
    score = cross_val_score(classifier_obj, features, target, scoring='f1', n_jobs=-1, cv=5)
    accuracy = score.mean()
    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=10)
    print(study.best_trial)

Лучший результат с гиперпараметрами max_depth=16, rf_n_estimators=46.

In [ ]:
model_forest = RandomForestClassifier(max_depth=16, n_estimators=37, random_state=12345)
model_forest.fit(features_train, target_train)
predictions_forest = model_forest.predict(features_test)
f1_forest = f1_score(target_test, predictions_forest)
print('F1 модели случайного леса:', '{:.2f}'.format(f1_forest))

### Модель LightGBM

In [ ]:
model_gbm = LGBMClassifier(random_state=12345, class_weight='balanced')
model_gbm.fit(features_train, target_train, verbose=10)

predictions_gbm = model_gbm.predict(features_test)
f1_gbm = f1_score(target_test, predictions_gbm)

print('F1 модели градиентного бустинга библиотеки lightGBM:', '{:.2f}'.format(f1_gbm))

### Модель CatBoost

In [ ]:
model_cat = CatBoostClassifier(iterations=100, random_seed=12345)
model_cat.fit(features_train, target_train, verbose=10)

predictions_cat = model_cat.predict(features_test)
f1_cat = f1_score(target_test, predictions_cat)

print('F1 модели градиентного бустинга библиотеки CatBoost:', '{:.2f}'.format(f1_cat))

## Выводы